### 方策の関数近似
REINFORCE \\
A3C \\
A2C \\
DDPG \\
それぞれ実装していこうかと思います。

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import gym

# 強化学習環境の設定
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [14]:
class PolicyNetwork(nn.Module):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 64)
        self.fc2 = nn.Linear(64, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.softmax(x, dim=-1)

In [15]:
policy_net = PolicyNetwork()
optimizer = optim.Adam(policy_net.parameters(), lr=0.01)


In [18]:
def update_policy(episode_states, episode_actions, episode_rewards):
    discounted_rewards = []
    running_add = 0
    for r in episode_rewards[::-1]:
        running_add = running_add * 0.99 + r
        discounted_rewards.append(running_add)
    discounted_rewards = torch.tensor(discounted_rewards[::-1])

    # NumPyの配列をPyTorchのテンソルに変換
    episode_states = torch.tensor(episode_states, dtype=torch.float32)
    episode_actions = torch.tensor(episode_actions, dtype=torch.int64)

    policy_probs = policy_net(episode_states)
    chosen_action_probs = torch.gather(policy_probs, 1, episode_actions.unsqueeze(1)).squeeze()

    loss = -torch.sum(torch.log(chosen_action_probs) * discounted_rewards)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [19]:
num_episodes = 1000
episode_states, episode_actions, episode_rewards = [], [], []

for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0

    while True:
        action_probs = policy_net(torch.tensor(state).float())
        action = torch.multinomial(action_probs, 1).item()

        next_state, reward, done, _ = env.step(action)

        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)

        total_reward += reward
        state = next_state

        if done:
            break

    # 方策の更新
    update_policy(episode_states, episode_actions, episode_rewards)

    if episode % 10 == 0:
        print(f"Episode {episode}: Total Reward: {total_reward}")

<ipython-input-18-5c94b1c07be4>:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  episode_states = torch.tensor(episode_states, dtype=torch.float32)


Episode 0: Total Reward: 13.0
Episode 10: Total Reward: 16.0
Episode 20: Total Reward: 13.0
Episode 30: Total Reward: 22.0
Episode 40: Total Reward: 13.0
Episode 50: Total Reward: 15.0
Episode 60: Total Reward: 13.0
Episode 70: Total Reward: 18.0
Episode 80: Total Reward: 19.0
Episode 90: Total Reward: 19.0
Episode 100: Total Reward: 13.0
Episode 110: Total Reward: 13.0
Episode 120: Total Reward: 14.0
Episode 130: Total Reward: 15.0
Episode 140: Total Reward: 12.0
Episode 150: Total Reward: 14.0
Episode 160: Total Reward: 15.0
Episode 170: Total Reward: 12.0
Episode 180: Total Reward: 26.0
Episode 190: Total Reward: 14.0
Episode 200: Total Reward: 20.0
Episode 210: Total Reward: 9.0
Episode 220: Total Reward: 25.0
Episode 230: Total Reward: 11.0
Episode 240: Total Reward: 14.0
Episode 250: Total Reward: 14.0
Episode 260: Total Reward: 14.0
Episode 270: Total Reward: 16.0
Episode 280: Total Reward: 10.0
Episode 290: Total Reward: 12.0
Episode 300: Total Reward: 12.0
Episode 310: Total R